In [1]:
import pandas as pd
import numpy as np
import xlearn as xl

Считываем данные и сразу дропаем ненужные фичи. Как и в первой домашке беру рандомно один процент данных для ускорения и потому что xlearn.cv не считает метрики на всём датасете: показывает -nan(ind).

In [2]:
data = pd.read_csv('../data/data.csv').sample(frac=0.01, random_state=1337)
data.drop(['banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1'], axis=1, inplace=True)
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,impressions,clicks
741246,2021-09-29 18:06:03.000000,13,358,9147519693626110830,0,2,5,1,0
11133560,2021-09-26 02:04:47.000000,0,3,6359232853395132451,0,2,1,1,0
5348767,2021-09-28 00:26:59.000000,15,16,794521774265082201,16,0,6,1,0
4078254,2021-09-26 17:07:20.000000,17,59,3250419304175957434,0,2,7,1,0
8784861,2021-09-29 15:47:26.000000,17,22,2904848336510916961,0,2,7,1,0


Пользуемся анализом из прошлой задачи + проверяем, что в новом поле нет нулов и смотрим на количество разных юзеров

In [3]:
def analysis(data: pd.DataFrame):
    print(f"Any nulls in oaid_hash={data['oaid_hash'].isnull().values.any()}")
    print(f"Number of users={data['oaid_hash'].nunique()}")
    data['date_time'] = pd.to_datetime(data['date_time'])

analysis(data)

Any nulls in oaid_hash=False
Number of users=148281


Эта часть полностью дублирует прошлую задачу: удаляем campaign_clicks и impressions. И разделяем на train и test.

In [4]:
def feature_engineering(data: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame):
    data.drop(['campaign_clicks', 'impressions'], axis=1, inplace=True)
    train = data[data['date_time'] < '2021-10-02']
    train_clicks = train['clicks']
    test = data[data['date_time'] >= '2021-10-02']
    test_clicks = test['clicks']
    train = train.drop(['date_time', 'clicks'], axis=1)
    test = test.drop(['date_time', 'clicks'], axis=1)
    return train, train_clicks, test, test_clicks

train, train_clicks, test, test_clicks = feature_engineering(data)

Факторизуем все столбцы, так как все фичи у нас категориальные

In [5]:
def factor_encoding(train, test):
    full = pd.concat([train, test], axis=0, sort=False)
    # Factorize everything
    for f in full:
        full[f], _ = pd.factorize(full[f])
        full[f] += 1

    return full.iloc[:train.shape[0]], full.iloc[train.shape[0]:]

train_f, test_f = factor_encoding(train, test)

Приводим к libffm формату: эту клетку я взял с одного ноутбука из kaggle. Сохраняем train и test в файлы.

In [6]:
class LibFFMEncoder(object):
    def __init__(self):
        self.encoder = 1
        self.encoding = {}

    def encode_for_libffm(self, row):
        txt = f"{row[0]}"
        for i, r in enumerate(row[1:]):
            try:
                txt += f' {i+1}:{self.encoding[(i, r)]}:1'
            except KeyError:
                self.encoding[(i, r)] = self.encoder
                self.encoder += 1
                txt += f' {i+1}:{self.encoding[(i, r)]}:1'

        return txt

encoder = LibFFMEncoder()
libffm_format_trn = pd.concat([train_clicks, train_f], axis=1).apply(
        lambda row: encoder.encode_for_libffm(row), raw=True, axis=1
)
libffm_format_tst = pd.concat([test_clicks, test_f], axis=1).apply(
    lambda row: encoder.encode_for_libffm(row), raw=True, axis=1
)

libffm_format_trn.to_csv(f'train.txt', index=False, header=False)
libffm_format_tst.to_csv(f'test.txt', index=False, header=False)

Подбираем гиперпараметры: регуляризацию и размерность эмбедингов.

In [7]:
for k in [2, 4, 8, 16]:
    for c in [0.000001, 0.00001, 0.0001, 0.001, 0.01]:
        ffm_model = xl.create_ffm()
        ffm_model.setTrain("./train.txt")
        param = {'task':'binary', 'lr':0.2, 'lambda': c, 'k': k, 'metric': 'auc', 'fold': 4}
        print(f'K={k}, C={c}')
        ffm_model.cv(param)

K=2, C=1e-06
K=2, C=1e-05
K=2, C=0.0001
K=2, C=0.001
K=2, C=0.01
K=4, C=1e-06
K=4, C=1e-05
K=4, C=0.0001
K=4, C=0.001
K=4, C=0.01
K=8, C=1e-06
K=8, C=1e-05
K=8, C=0.0001
K=8, C=0.001
K=8, C=0.01
K=16, C=1e-06
K=16, C=1e-05
K=16, C=0.0001
K=16, C=0.001
K=16, C=0.01


Сравниваем с бейзлайном: побили его и предыдущую модель.

In [8]:
from sklearn.metrics import roc_auc_score, log_loss

ffm_model = xl.create_ffm()
ffm_model.setTrain('train.txt')
ffm_model.setValidate('test.txt')
param = {'task':'binary', 'lr':0.2, 'lambda':0.0001, 'k': 8, 'metric': 'auc'}

ffm_model.fit(param, './model.out')

ffm_model.setSigmoid()
ffm_model.setTest('test.txt')
ffm_model.predict('./model.out', './output.txt')

with open('output.txt', 'r') as f:
    y_pred_proba = np.array(list(map(float, filter(lambda s: len(s) > 0, f.read().split('\n')))))
roc_auc_metric = roc_auc_score(test_clicks, y_pred_proba)
log_loss_metric = log_loss(test_clicks, y_pred_proba)
print(f"roc_auc={roc_auc_metric}, log_loss={log_loss_metric}")

y_pred_base = np.full(y_pred_proba.shape, np.mean(train_clicks))
roc_auc_metric_base = roc_auc_score(test_clicks, y_pred_base)
log_loss_metric_base = log_loss(test_clicks, y_pred_base)
print(f"roc_auc_base: {roc_auc_metric_base}, log_loss base: {log_loss_metric_base}")

roc_auc=0.7769001506067655, log_loss=0.14258316108167948
roc_auc_base: 0.5, log_loss base: 0.16643236596578298
